<a href="https://colab.research.google.com/github/Muditha98/keras_notebooks/blob/main/keras_learning_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras

#### Training a convolutional network to classify MNIST digits

In [17]:
# Load the data and split it between train and test sets
(x_train, y_train),(x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to [0,1] range
x_train = x_train.astype("float32") /255
x_test = x_test.astype("float32") / 255

# Make sure iages have shape (28,28,1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape: ",x_train.shape)
print("y_train shape: ",y_train.shape)

print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape:  (60000, 28, 28, 1)
y_train shape:  (60000,)
60000 train samples
10000 test samples


#### Modle building

In [22]:
num_classes = 10
input_shape = (28,28,1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
        keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
        keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax")
    ]
)

In [23]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)                   │ (None, 26, 26, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 24, 24, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 10, 10, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name='acc'),
    ],
)

In [29]:
batch_size = 128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)

score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - acc: 0.5310 - loss: 1.3084 - val_acc: 0.9616 - val_loss: 0.1366
Epoch 2/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - acc: 0.9291 - loss: 0.2408 - val_acc: 0.9727 - val_loss: 0.0917
Epoch 3/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - acc: 0.9514 - loss: 0.1629 - val_acc: 0.9806 - val_loss: 0.0624
Epoch 4/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - acc: 0.9617 - loss: 0.1291 - val_acc: 0.9861 - val_loss: 0.0487
Epoch 5/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - acc: 0.9675 - loss: 0.1070 - val_acc: 0.9851 - val_loss: 0.0544
Epoch 6/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.9731 - loss: 0.0920 - val_acc: 0.9892 - val_loss: 0.0399
Epoch 7/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.9774 - loss: 0.0800 - val_acc: 0.9906 - val_loss: 0.0322
Epoch 8/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.9772 - loss: 0.0764 - val_acc: 0.9903 - val_loss: 0.0323
Epoch 9/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 4s 11m

In [30]:
model.save("final_model.keras")

In [32]:
predictions = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
